Steps:

1. Collect 100 points in json format.
2. JSON Format:
{
    name: Name of the point
    title: Name-Direction
    signals: [[signal_strength, bssid]] List of signal strengths and bssids.
}
3. Then convert the json into pandas table with columns: [name, title, ...all the bssids] and rows to be the name, title and signal strengths of that particular location. If a bssid is absent in that location, we make its value as 0.

In [1]:
!pip install pywifi comtypes

In [2]:
from pywifi import PyWiFi
import pandas as pd
import json
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVR
import time as tm

In [3]:

wifi = PyWiFi()
iface = wifi.interfaces()[0]
def scan_wifi(name, time, direction):

    # print("Scanning for Wi-Fi networks...")
    tm.sleep(2)
    access_points = iface.scan_results()
    location = {}
    for ap in access_points:
        # print(f"SSID: {ap.ssid}, BSSID: {ap.bssid}, RSSI: {ap.signal} dBm")
        location[ap.bssid] = ap.signal
    return {
        "name": name,
        "title": f"{name}-{time}-{direction}",
        "signals": location
    }

In [21]:
# Scanning the WiFis in the location.
names = []
directions = ["N", "E", "S", "W"]
time = "1207" # Date of collection MMDD
locations = {}

names = ['ELEVATOR FIRE EXTINGUISHER', 'ELEVATOR 1', 'ELEVATOR 2', 'ELEVATOR 3', '5100', '5102', '5101', '5104', '5103', '5105', '5106', '5107', '5107 RESTROOMS', '5108', '5108 RESTROOMS', '5109 RESTROOMS', '5111', '5112', '5113', '5114', '5119', '5115', '5117', '5118', '5115-5117 ENTRANCE', '5503', '5503 STAIRS ENTRANCE', '5360', '5400', '5360 2', '5359', '5358 1', '5357', '5356', '5355', '5358 2', '5354 QSA 1', '5354 QSA 2', '5354 QSA FE', '5352', '5353', '5351', '5350', '5349', '5348', '5347', '5346', '5349 2', '5345', '5344', '5344 TAPS', '5343', '5342', '5342 PANTRY 1', '5342 PANTRY 2', '5341', '5340', '5339 QSA ENTR', '5339 QSA 1', '5339 QSA 2', '5337', '5336', '5335', '5333', '5334', '5332', '5331', '5328', '5328 QSA 1', '5328 QSA 2', '5327', '5326', '5324', '5325', '5323', '5322', '5321', '5320', '5319', '5318', '5317', '5315', '5316', '5316 CR', '5309 FIRE EXTINGUISHER', '5309 STUDY AREA 1', '5309 STUDY AREA 2', '5309', '5308', '5307', '5306', '5305', '5304', '5303', '5302', '5201', '5309 TAPS', '5309 MEN RR',]

resp = ""
for name in names:
    clear_output(wait=True)
    for direction in directions:
        print(f"Scan {name} {direction}?")
        resp = input()
        if resp == 'qwertyuiop':
            break
        if resp == 'skip':
            break
        locations[f"{name}-{time}-{direction}"] = scan_wifi(name, time, direction)
        with open("raw_points.json", "w") as outfile: 
            json.dump(locations, outfile)
        print(f"Got {len(locations[f'{name}-{time}-{direction}']['signals'])} signals.\n")
    if resp == 'qwertyuiop':
            break
    if resp == 'skip':
        continue

print(locations)

Scan ELEVATOR FIRE EXTINGUISHER N?
{}


# DO NOT GO DOWN

# ----------------------------------------------------------------

# ----------------------------------------------------------------

# ----------------------------------------------------------------

# ----------------------------------------------------------------

# ----------------------------------------------------------------

# ----------------------------------------------------------------

In [15]:
# Convert the JSON to Table

selected = {}

with open("nguyen_selected_wifis.json", "r") as outfile: 
    selected = json.load(outfile)

# with open("nguyen_fselected_wifis.json", "r") as outfile: 
#     selected["bssids"] = json.load(outfile)["bssids"]

bssids = selected["bssids"]
ssids = selected["ssids"]
mapping = selected["mapping"]

raw_points = {}

with open("nguyen_raw_points.json", "r") as outfile: 
    raw_points = json.load(outfile)

# print(raw_points)

points = []

for i in raw_points.values():
    point = {
        "name": i["name"],
        "title": i["title"],
    }
    for j in bssids:
        point[j] = 0
    for j in i["signals"].keys():
        if j in bssids:
            point[j] = i["signals"][j]
    points.append(point)
            
points_table = pd.DataFrame(points, columns=points[0].keys()).iloc[:98*3]
# points_table = pd.DataFrame(points, columns=points[0].keys())
points_table.to_csv("raw_points.csv", index=False)
points_table

,name,title,70:69:5a:aa:4e:4e:,b0:26:80:83:32:ce:,70:69:5a:a6:40:21:,70:69:5a:a6:11:21:,b0:26:80:83:32:c1:,70:69:5a:aa:4e:41:,70:69:5a:aa:48:41:,70:69:5a:aa:48:4e:,...,70:69:5a:a6:3b:00:,b0:26:80:a9:fb:e0:,70:69:5a:a6:47:e0:,70:69:5a:92:f7:80:,70:69:5a:6e:a3:41:,70:69:5a:6e:a3:4e:,70:69:5a:6e:a3:4d:,70:69:5a:6e:a3:42:,70:69:5a:6e:a3:4f:,70:69:5a:6e:a3:40:
0,5101,5101-AFT-S,-87,-95,-86,0,0,0,0,0,...,0,0,0,-82,0,0,0,0,0,0
1,5101,5101-AFT-E,-87,-95,-86,0,0,0,0,0,...,0,0,0,-82,0,0,0,0,0,0
2,5101,5101-AFT-N,-85,-95,-89,0,0,0,0,0,...,0,0,0,-80,0,0,0,0,0,0
3,5101,5101-AFT-W,-85,-95,-89,0,0,0,0,0,...,0,0,0,-80,0,0,0,0,0,0
4,5102,5102-AFT-W,-85,-95,-89,0,0,0,0,0,...,0,0,0,-80,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,5325,5325-AFT-W,-54,0,-54,-75,0,0,0,-80,...,0,0,0,0,0,0,0,0,0,0
290,5325,5325-AFT-S,-54,0,-54,-75,0,0,0,-80,...,0,0,0,0,0,0,0,0,0,0
291,5324,5324-AFT-N,-54,0,-54,-75,0,0,0,-80,...,0,0,0,0,0,0,0,0,0,0
292,5324,5324-AFT-W,-54,0,-54,-75,0,0,0,-80,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Train Random Forest

X = points_table.drop(columns=["name", "title"])
y = points_table["name"]
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
mapping = dict(zip( range(len(label_encoder.classes_)), label_encoder.classes_))

# Create a Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X, y)

# svm_model = SVR(kernel='linear', C=1.0, epsilon=0.2)

# Train the model
# svm_model.fit(X, y)

RandomForestRegressor(random_state=42)

In [20]:
# Get the current location's WiFis and calculate the point.

M = 5

def calc_distance(s,d):
#     print(s["title"], d["title"])
    dist = 0
    for b in bssids:
#         print(b, s[b], d[b])
        dist += (d[b]-s[b])**2
#     print("\n\n\n")
    return dist**(1/2)

cpoint = scan_wifi("test", "test", "test")

point = {
    "name": cpoint["name"],
    "title": cpoint["title"],
}

for j in bssids:
    point[j] = 0
for j in cpoint["signals"].keys():
    if j in bssids:
        point[j] = cpoint["signals"][j]
cpoint = point

print(cpoint)
df = pd.DataFrame({k: [v] for k, v in cpoint.items()}).drop(columns=["name", "title"])
# print(df)

dists = []
for s in points:
    dists.append([calc_distance(s, cpoint), s["name"], s["title"]])
    
dists.sort()
dists = dists[:4*M+1]
print("*"*100)
print()
mode = []
mean = {}
for i in dists:
    print(i)
    mode.append(i[1])
    if i[1] in mean:
        mean[i[1]][0] += i[0]
        mean[i[1]][1] += 1
    else:
        mean[i[1]] = [i[0], 1]
for k,v in mean.items():
    mean[k] = mean[k][0]/mean[k][1]
mean = dict(sorted(mean.items(), key=lambda item: item[1]))
print()
print(mean)
print()
mean = list(mean.keys())[0]
print(f"Knn\nMode: {max(set(mode[:4*M+1]), key=mode[:4*M+1].count)}\nMean: {mean}")
print(f"\nRandom Forest: {mapping[rf_model.predict(df)[0]//1]}\n")
# print(f"\nsvm: {mapping[svm_model.predict(df)[0]//1]}\n")

{'name': 'test', 'title': 'test-test-test', '70:69:5a:aa:4e:4e:': 0, 'b0:26:80:83:32:ce:': 0, '70:69:5a:a6:40:21:': -76, '70:69:5a:a6:11:21:': -71, 'b0:26:80:83:32:c1:': -68, '70:69:5a:aa:4e:41:': -76, '70:69:5a:aa:48:41:': -59, '70:69:5a:aa:48:4e:': -76, '70:69:5a:aa:4e:4d:': 0, 'b0:26:80:83:32:cd:': 0, '70:69:5a:a6:11:22:': -69, '70:69:5a:a6:40:22:': -67, 'b0:26:80:83:32:c2:': -67, '70:69:5a:aa:4e:42:': -76, '70:69:5a:aa:48:42:': -76, '70:69:5a:aa:48:4d:': -70, '70:69:5a:a6:3a:8f:': -90, 'b0:26:80:83:32:cf:': 0, '70:69:5a:a6:40:20:': -74, '70:69:5a:a6:11:20:': -73, 'b0:26:80:83:32:c0:': -62, '70:69:5a:aa:4e:40:': -77, '70:69:5a:aa:48:40:': 0, '70:69:5a:aa:48:4f:': -75, '70:69:5a:aa:4e:4f:': 0, '70:69:5a:a6:40:2e:': -80, '70:69:5a:a6:11:2e:': -78, '70:69:5a:a6:40:2f:': -80, '70:69:5a:a6:11:2f:': -80, '70:69:5a:a6:11:2d:': -77, 'b0:26:80:c9:7b:6e:': -86, 'b0:26:80:83:37:0e:': -79, 'b0:26:80:c9:7b:61:': -79, 'b0:26:80:83:37:01:': 0, 'b0:26:80:aa:70:a1:': -89, 'b0:26:80:c9:7b:6d:': -84, 

# Cross Validation

In [21]:
locs_map = [
    {"name": "5339 QSA 1", "x": 51, "y": 337},
    {"name": "5328 QSA 2", "x": 51, "y": 34},
    {"name": "5339 QSA 2", "x": 29, "y": 337},
    {"name": "5339 QSA ENTR", "x": 41, "y": 294},
    {"name": "5336", "x": 90, "y": 251},
    {"name": "5334", "x": 90, "y": 171},
    {"name": "5337", "x": 81, "y": 262},
    {"name": "5335", "x": 81, "y": 222},
    {"name": "5333", "x": 81, "y": 182},
    {"name": "5332", "x": 81, "y": 148},
    {"name": "5331", "x": 81, "y": 116},
    {"name": "5328", "x": 64, "y": 84},
    {"name": "5327", "x": 94, "y": 84},
    {"name": "5326", "x": 129, "y": 84},
    {"name": "5325", "x": 165, "y": 84},
    {"name": "5324", "x": 171, "y": 90},
    {"name": "5322", "x": 186, "y": 90},
    {"name": "5321", "x": 237, "y": 90},
    {"name": "5317", "x": 319, "y": 90},
    {"name": "5316", "x": 340, "y": 90},
    {"name": "5316 CR", "x": 350, "y": 120},
    {"name": "5309 FIRE EXTINGUISHER", "x": 392, "y": 79},
    {"name": "5309", "x": 405, "y": 90},
    {"name": "5306", "x": 458, "y": 90},
    {"name": "5201", "x": 591, "y": 90},
    {"name": "5302", "x": 582, "y": 83},
    {"name": "5303", "x": 547, "y": 83},
    {"name": "5304", "x": 512, "y": 83},
    {"name": "5305", "x": 476, "y": 83},
    {"name": "5307", "x": 443, "y": 83},
    {"name": "5308", "x": 409, "y": 83},
    {"name": "5309 TAPS", "x": 386, "y": 143},
    {"name": "5309 MEN RR", "x": 386, "y": 193},
    {"name": "5323", "x": 201, "y": 84},
    {"name": "5320", "x": 233, "y": 84},
    {"name": "5319", "x": 268, "y": 84},
    {"name": "5318", "x": 305, "y": 84},
    {"name": "5315", "x": 337, "y": 84},
    {"name": "5328 QSA 1", "x": 29, "y": 34},
    {"name": "ELEVATOR FIRE EXTINGUISHER", "x": 658, "y": 255},
    {"name": "ELEVATOR 1", "x": 658, "y": 225},
    {"name": "ELEVATOR 2", "x": 658, "y": 185},
    {"name": "ELEVATOR 3", "x": 658, "y": 145},
    {"name": "5100", "x": 658, "y": 91},
    {"name": "5102", "x": 688, "y": 85},
    {"name": "5101", "x": 708, "y": 85},
    {"name": "5104", "x": 738, "y": 85},
    {"name": "5105", "x": 768, "y": 85},
    {"name": "5103", "x": 748, "y": 95},
    {"name": "5106", "x": 797, "y": 109},
    {"name": "5107", "x": 815, "y": 140},
    {"name": "5107 RESTROOMS", "x": 822, "y": 160},
    {"name": "5109", "x": 836, "y": 175},
    {"name": "5109 RESTROOMS", "x": 845, "y": 198},
    {"name": "5108", "x": 830, "y": 164},
    {"name": "5108 RESTROOMS", "x": 834, "y": 183},
    {"name": "5112", "x": 858, "y": 209},
    {"name": "5111", "x": 857, "y": 220},
    {"name": "5113", "x": 873, "y": 236},
    {"name": "5114", "x": 941, "y": 240},
    {"name": "5119", "x": 930, "y": 257},
    {"name": "5115", "x": 810, "y": 257},
    {"name": "5118", "x": 740, "y": 257},
    {"name": "5115-5117 ENTRANCE", "x": 714, "y": 256},
    {"name": "5503", "x": 700, "y": 248},
    {"name": "5503 STAIRS ENTRANCE", "x": 684, "y": 248},
    {"name": "5360", "x": 614, "y": 248},
    {"name": "5360 2", "x": 519, "y": 248},
    {"name": "5400", "x": 589, "y": 259},
    {"name": "5359", "x": 514, "y": 283},
    {"name": "5358 1", "x": 492, "y": 273},
    {"name": "5358 2", "x": 412, "y": 273},
    {"name": "5353", "x": 346, "y": 283},
    {"name": "5352", "x": 362, "y": 273},
    {"name": "5350", "x": 314, "y": 283},
    {"name": "5351", "x": 330, "y": 273},
    {"name": "5348", "x": 281, "y": 283},
    {"name": "5349", "x": 297, "y": 273},
    {"name": "5349 2", "x": 214, "y": 273},
    {"name": "5344", "x": 181, "y": 273},
    {"name": "5347", "x": 247, "y": 283},
    {"name": "5346", "x": 214, "y": 283},
    {"name": "5345", "x": 177, "y": 283},
    {"name": "5343", "x": 144, "y": 283},
    {"name": "5344 TAPS", "x": 166, "y": 273},
    {"name": "5342", "x": 151, "y": 273},
    {"name": "5342 PANTRY 1", "x": 154, "y": 251},
    {"name": "5342 PANTRY 2", "x": 169, "y": 251},
    {"name": "5341", "x": 111, "y": 283},
    {"name": "5340", "x": 81, "y": 283},
    {"name": "5355", "x": 418, "y": 283},
    {"name": "5354 QSA 1", "x": 392, "y": 336},
    {"name": "5309 STUDY AREA 2", "x": 392, "y": 34},
    {"name": "5354 QSA 2", "x": 372, "y": 336},
    {"name": "5309 STUDY AREA 1", "x": 372, "y": 34},
    {"name": "5354 QSA FE", "x": 392, "y": 298},
    {"name": "5356", "x": 448, "y": 283},
    {"name": "5357", "x": 481, "y": 283},
    {"name": "5117", "x": 770, "y": 257}
    ]
t = {}
for x in locs_map:
    t[x["name"]] = [x["x"], x["y"]]
locs_map = t
print(locs_map)

{'5339 QSA 1': [51, 337], '5328 QSA 2': [51, 34], '5339 QSA 2': [29, 337], '5339 QSA ENTR': [41, 294], '5336': [90, 251], '5334': [90, 171], '5337': [81, 262], '5335': [81, 222], '5333': [81, 182], '5332': [81, 148], '5331': [81, 116], '5328': [64, 84], '5327': [94, 84], '5326': [129, 84], '5325': [165, 84], '5324': [171, 90], '5322': [186, 90], '5321': [237, 90], '5317': [319, 90], '5316': [340, 90], '5316 CR': [350, 120], '5309 FIRE EXTINGUISHER': [392, 79], '5309': [405, 90], '5306': [458, 90], '5201': [591, 90], '5302': [582, 83], '5303': [547, 83], '5304': [512, 83], '5305': [476, 83], '5307': [443, 83], '5308': [409, 83], '5309 TAPS': [386, 143], '5309 MEN RR': [386, 193], '5323': [201, 84], '5320': [233, 84], '5319': [268, 84], '5318': [305, 84], '5315': [337, 84], '5328 QSA 1': [29, 34], 'ELEVATOR FIRE EXTINGUISHER': [658, 255], 'ELEVATOR 1': [658, 225], 'ELEVATOR 2': [658, 185], 'ELEVATOR 3': [658, 145], '5100': [658, 91], '5102': [688, 85], '5101': [708, 85], '5104': [738, 85

In [73]:
# KNN

M = 3

def calc_distance(s,d):
    dist = 0
    for b in bssids:
        dist += (d[b]-s[b])**2
    return dist**(1/2)

def calc_error_dist(x, y):
    x = locs_map[x]
    y = locs_map[y]
    dist = (x[0]-y[0])**2 + (x[1]-y[1])**2 
    return dist**(1/2)

def predict(test_df, loc):
    dists = []
    for i in range(len(test_df)):
        s = test_df.iloc[i]
        dists.append([calc_distance(s, loc), s["name"], s["title"]])
    dists.sort()
    dists = dists[:4*M+1]
    for i in range(len(dists)):
        dists[i] = dists[i][1]
    return max(set(dists), key=dists.count)

X_train, X_test = train_test_split(points_table, test_size=0.2, random_state=42)

predictions = []
avg = 0
mse = 0
for i in range(len(X_test)):
    loc = X_test.iloc[i]
    prediction = predict(X_train, loc)
    e = calc_error_dist(loc['name'], prediction)
    mse += e

    if e <= 50:
        es = "Good"
        avg+=1
    elif e > 50 and e < 100:
        es = "Okay" 
        avg+=1//2
    else:
        es = "Bad"

    # print(f"{loc['name']} - {prediction} - {loc['name'] == prediction}")
    predictions.append([loc['name'], prediction, e, es])

n = len(predictions)

print(f"Accuracy = {avg/n}\nMSE = {mse/n}\n")
with open("predictions.json", "w") as file:
    json.dump({"predictions": predictions}, file)

Accuracy = 0.6440677966101694
MSE = 56.48040461544177



In [70]:
# Random Forest

X = points_table.drop(columns=["name", "title"])
y = points_table["name"]
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 72.0653675724015


In [71]:
#SVM

from sklearn.svm import SVR

# Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an SVM Regressor
svm_model = SVR(kernel='linear', C=1.0, epsilon=0.2)

# Train the model
svm_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 290.61282372965866


In [78]:

# KNN

M = 3

def calc_distance(s,d):
    dist = 0
    for b in bssids:
        dist += (d[b]-s[b])**2
    return dist**(1/2)

def calc_error_dist(x, y):
    x = locs_map[x]
    y = locs_map[y]
    dist = (x[0]-y[0])**2 + (x[1]-y[1])**2 
    return dist**(1/2)

def predict(test_df, loc):
    dists = []
    for i in range(len(test_df)):
        s = test_df.iloc[i]
        dists.append([calc_distance(s, loc), s["name"], s["title"]])
    dists.sort()
    dists = dists[:4*M+1]
    for i in range(len(dists)):
        dists[i] = dists[i][1]
    return max(set(dists), key=dists.count)

X_train, X_test = train_test_split(points_table, test_size=0.2, random_state=42)

predictions = []
avg = 0
mse = 0
for i in range(len(X_test)):
    loc = X_test.iloc[i]
    prediction = predict(X_train, loc)
    e = calc_error_dist(loc['name'], prediction)
    mse += e

    if e <= 95:
        es = "Good"
        avg+=1
    elif e > 95 and e < 100:
        es = "Okay" 
        avg+=1//2
    else:
        es = "Bad"

    # print(f"{loc['name']} - {prediction} - {loc['name'] == prediction}")
    predictions.append([loc['name'], prediction, e, es])

n = len(predictions)

print(f"Accuracy = {avg/n}\nMSE = {mse/n}\n")
with open("predictions.json", "w") as file:
    json.dump({"predictions": predictions}, file)


Accuracy = 0.8220338983050848
MSE = 56.48040461544177

